In [0]:
#Reading: https://blog.paperspace.com/detecting-and-localizing-pneumonia-from-chest-x-ray-scans-with-pytorch/
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

In [1]:
! git clone https://github.com/vicely07/COVID19-image-classification.git

Cloning into 'COVID19-image-classification'...
remote: Enumerating objects: 270, done.
remote: Total 270 (delta 0), reused 0 (delta 0), pack-reused 270
Receiving objects: 100% (270/270), 235.91 MiB | 38.48 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Checking out files: 100% (252/252), done.


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_dir="/content/COVID19-image-classification/Data/all/test"

In [21]:
from keras.models import load_model
import sys
import os
from keras.models import model_from_json
import numpy as np


json_file = open('/content/COVID19-image-classification/Best_weights/covid19_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model = load_model('/content/COVID19-image-classification/Best_weights/Covid_Multi.h5')
BATCH_SIZE = 10 
IMAGE_SIZE = (150, 150)

test_datagen = ImageDataGenerator(rescale=1. / 255)
eval_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=IMAGE_SIZE,
        batch_size=1,
        shuffle=True,
        seed=42,
        class_mode="categorical")
eval_generator.reset()    

x = model.evaluate_generator(eval_generator,
                           use_multiprocessing = False,
                           verbose = 1,
                           workers=1
                           )


print('Test loss:' , x[0])
print('Test accuracy:',x[1])

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


Found 27 images belonging to 3 classes.
27/27 [==============================] - 9s 339ms/step
Test loss: 0.3542379740037076
Test accuracy: 0.8888888888888888


In [0]:
eval_generator.filenames

In [0]:
import pandas as pd
pred=model.predict_generator(eval_generator,verbose=1, steps=27)
print(len(pred))
predicted_class_indices=np.argmax(pred,axis=1)
labels = {0: 'covid', 1: 'normal', 2: 'pneumonia_bac'}
predictions = [labels[k] for k in predicted_class_indices]
acc=np.max(pred, axis=1)*100
loss=np.min(pred, axis=1)*100
filenames=eval_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions,
                      "Accuracy": acc,
                      "Loss": loss})
results["Filename"] = results["Filename"].apply(lambda x: str(x).split("/")[1])
results["Predictions"] = results["Predictions"].apply(lambda x: str(x).upper())
results["Accuracy"] = results["Accuracy"].apply(lambda x: str(round(x, 2))+"%")
results["Loss"] = results["Loss"].apply(lambda x: str(round(x, 2))+"%")
results.to_csv("/content/COVID19-image-classification/Output_Image/Output_pred.csv")

In [0]:
results

In [0]:
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

files = results["Filename"]
predictions = results["Predictions"]
acc = results["Accuracy"] 
loss = results["Loss"]
f, ax = plt.subplots(4,3, figsize=(40, 20))
for i in range(len(files)-15):
  img_path = eval_generator.filepaths[i]
  img = load_img(img_path, target_size=(150, 150))
  file_name = files[i]
  ax.ravel()[i].imshow(img, cmap='gray')
  ax.ravel()[i].set_title(predictions[i]+"-Acc: "+acc[i]+"-Loss: "+loss[i], fontsize=20)
  plt.tight_layout()
plt.savefig('/content/COVID19-image-classification/Output_Image/test_batch_2.png')

In [0]:
from google.colab import files
files.download('/content/COVID19-image-classification/Output_Image/Output_pred.csv')
#files.download('/content/COVID19-image-classification/Output_Image/test_batch_2.png')